In [2]:
import arcpy
import os
import rasterio as rio
from osgeo import gdal
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import seaborn as sns
import geopandas
import matplotlib.pyplot as plt
import rasterio
import rasterio.plot
import matplotlib.colors as colors

In [6]:
path='E:\program'
os.chdir(path)

In [ ]:
inputFolder=r'rainfall_data'
for file in os.listdir(inputFolder):
    input_file=os.path.join(r'rainfall_data', file)
    output_file=os.path.join(r'clipped_rasters', file)
    with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_UTM_Zone_43N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",75.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', extent='259023 2061053 338623 2162253 PROJCS["WGS_1984_UTM_Zone_43N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",75.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
        arcpy.management.Clip(input_file, "259023 2061053 338623 2162253", output_file, "mmr_landuse_2016_final.tif", "-1", "NONE", "NO_MAINTAIN_EXTENT")

In [4]:
inputFolder=r'clipped_rasters'
for file in os.listdir(inputFolder):
    if file.endswith('tif'):
        input_file=os.path.join(r'clipped_rasters', file)
        output_file=os.path.join(r'final_rasters', file)
        with arcpy.EnvManager(outputCoordinateSystem='PROJCS["WGS_1984_UTM_Zone_43N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",75.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', extent='259023 2061053 338623 2162253 PROJCS["WGS_1984_UTM_Zone_43N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",75.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
            arcpy.management.Resample(input_file, output_file, "200 200", "BILINEAR")

In [6]:
inputFolder=r'final_rasters'
for file in os.listdir(inputFolder):
    if file.endswith('tif'):
        print(file)

inunriver_rcp4p5_00000NorESM1-M_2050_rp00002.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00010.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00005.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00025.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00050.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00100.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00250.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp00500.tif
inunriver_rcp4p5_00000NorESM1-M_2050_rp01000.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00002.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00005.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00010.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00025.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00050.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00100.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00250.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00500.tif
inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp01000.tif
inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00002.tif
inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00005.tif
inunriver_rcp4p5_000

In [18]:
def func(inputFolder):
    j=0
    df=pd.DataFrame(columns=['fraction inundated','average inundation','name'], index=['2','5','10','25','50','100','250','500','1000'])
    for i in range(45):
        file=os.path.join(inputFolder,os.listdir(inputFolder)[i])
        if file.endswith('tif'):
            with rio.open(file) as dem_src:
                raster_file = dem_src.read(1)
                raster = gdal.Open(file)
                projection = raster.GetProjection()
                geotransform = raster.GetGeoTransform()
                df.iloc[j,0]=(raster_file > 0).sum()/(len(raster_file)*len(raster_file[1]))
                df.iloc[j,1]=(raster_file[raster_file>0]).mean()
                df.iloc[j,2]=os.listdir(inputFolder)[i]
                j+=1
    return df

In [19]:
df_1_low=func(r'final_rasters\model_1_low')
df_2_low=func(r'final_rasters\model_2_low')
df_3_low=func(r'final_rasters\model_3_low')
df_4_low=func(r'final_rasters\model_4_low')
df_5_low=func(r'final_rasters\model_5_low')

In [20]:
df_1_low

,fraction inundated,average inundation,name
2,0.088612,0.147269,inunriver_rcp4p5_00000NorESM1-M_2050_rp00002.tif
5,0.104533,0.475867,inunriver_rcp4p5_00000NorESM1-M_2050_rp00005.tif
10,0.11314,0.672428,inunriver_rcp4p5_00000NorESM1-M_2050_rp00010.tif
25,0.119866,0.921064,inunriver_rcp4p5_00000NorESM1-M_2050_rp00025.tif
50,0.123348,1.113517,inunriver_rcp4p5_00000NorESM1-M_2050_rp00050.tif
100,0.128081,1.281791,inunriver_rcp4p5_00000NorESM1-M_2050_rp00100.tif
250,0.131563,1.516608,inunriver_rcp4p5_00000NorESM1-M_2050_rp00250.tif
500,0.133675,1.691725,inunriver_rcp4p5_00000NorESM1-M_2050_rp00500.tif
1000,0.13508,1.870278,inunriver_rcp4p5_00000NorESM1-M_2050_rp01000.tif


In [22]:
df_2_low

,fraction inundated,average inundation,name
2,0.099479,0.389265,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00002.tif
5,0.117106,0.793279,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00005.tif
10,0.122834,1.064002,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00010.tif
25,0.129639,1.377687,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00025.tif
50,0.13279,1.618577,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00050.tif
100,0.134956,1.859926,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00100.tif
250,0.138681,2.151558,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00250.tif
500,0.142712,2.341141,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00500.tif
1000,0.146589,2.522868,inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp01000.tif


In [23]:
df_3_low

,fraction inundated,average inundation,name
2,0.095804,0.33736,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00002.tif
5,0.115588,0.742864,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00005.tif
10,0.122136,1.006358,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00010.tif
25,0.129125,1.368239,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00025.tif
50,0.132666,1.632243,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00050.tif
100,0.135609,1.887159,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00100.tif
250,0.140451,2.191374,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00250.tif
500,0.143849,2.411875,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00500.tif
1000,0.147465,2.617799,inunriver_rcp4p5_0000HadGEM2-ES_2050_rp01000.tif


In [24]:
df_4_low

,fraction inundated,average inundation,name
2,0.092604,0.247235,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00002.tif
5,0.112145,0.621538,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00005.tif
10,0.118674,0.864708,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00010.tif
25,0.125237,1.163498,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00025.tif
50,0.129639,1.385039,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00050.tif
100,0.132711,1.610664,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00100.tif
250,0.13554,1.907631,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00250.tif
500,0.138606,2.109,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00500.tif
1000,0.141673,2.30116,inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp01000.tif


In [25]:
df_5_low

,fraction inundated,average inundation,name
2,0.086031,0.032067,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00002.tif
5,0.095433,0.328136,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00005.tif
10,0.105345,0.487545,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00010.tif
25,0.113713,0.684798,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00025.tif
50,0.118189,0.829467,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00050.tif
100,0.121004,0.975501,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00100.tif
250,0.12499,1.155077,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00250.tif
500,0.128081,1.282958,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00500.tif
1000,0.129714,1.420665,inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp01000.tif


In [75]:
def func(z,jaar):
    all_models=[]
    for y in range(506):
        b= []
        for x in range(398):
            b.append([0,0,0,0,0])
        all_models.append(b)
        
    avg_models_low=[]
    for y in range(506):
        b= []
        for x in range(398):
            b.append(0)
        avg_models_low.append(b)
    avg_models_low=np.array(avg_models_low, dtype = float)

    for count, folder in enumerate(os.listdir(r'final_rasters\low')):
        folder2=os.path.join(r'final_rasters\low',folder)
        file=os.path.join(folder2,os.listdir(folder2)[z])
        print(file)
        if file.endswith('tif'):
            with rio.open(file) as dem_src:
                raster_file = dem_src.read(1)  
                for i in range(506):
                    for j in range(398):
                        all_models[i][j][count]=raster_file[i][j]
                        
    for i in range(506):
        for j in range(398):
            avg_models_low[i][j]=np.mean(all_models[i][j])

    with rio.open(r'mmr_landuse_2016_final.tif') as dem_src:
        raster = gdal.Open(file)
        projection = raster.GetProjection()
        geotrans = raster.GetGeoTransform()
    outputPath = os.path.join(r'combined_rasters','avg_models_low_'+ str(jaar) +'.tif')
    print(outputPath)
    cv2.imwrite(outputPath, avg_models_low)
    dataset2 = gdal.Open(outputPath, gdal.GA_Update)
    dataset2.SetGeoTransform(geotrans)
    dataset2.SetProjection(projection)
    return avg_models_low  

In [76]:
avg_models_low_2=func(1,2)
avg_models_low_5=func(6,5)
avg_models_low_10=func(11,10)
avg_models_low_25=func(16,25)
avg_models_low_50=func(21,50)
avg_models_low_100=func(26,100)
avg_models_low_250=func(31,250)
avg_models_low_500=func(36,500)
avg_models_low_1000=func(41,1000)

final_rasters\low\model_5_low\inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00002.tif
final_rasters\low\model_1_low\inunriver_rcp4p5_00000NorESM1-M_2050_rp00002.tif
final_rasters\low\model_3_low\inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00002.tif
final_rasters\low\model_4_low\inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00002.tif
final_rasters\low\model_2_low\inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00002.tif
combined_rasters\avg_models_low_2.tif
final_rasters\low\model_5_low\inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00005.tif
final_rasters\low\model_1_low\inunriver_rcp4p5_00000NorESM1-M_2050_rp00005.tif
final_rasters\low\model_3_low\inunriver_rcp4p5_0000HadGEM2-ES_2050_rp00005.tif
final_rasters\low\model_4_low\inunriver_rcp4p5_00IPSL-CM5A-LR_2050_rp00005.tif
final_rasters\low\model_2_low\inunriver_rcp4p5_0000GFDL-ESM2M_2050_rp00005.tif
combined_rasters\avg_models_low_5.tif
final_rasters\low\model_5_low\inunriver_rcp4p5_MIROC-ESM-CHEM_2050_rp00010.tif
final_rasters\low\model_1_low\inunriver_rcp4p5_00000Nor

### Export zoning low

In [77]:
file_path=r'mmr_landuse_2016_final.tif'
with rio.open(file_path) as dem_src:
    luc_mmr = dem_src.read(1)
    luc=gdal.Open(file_path)
    projection = luc.GetProjection()
    geotrans = luc.GetGeoTransform()

In [78]:
with rio.open(r'combined_rasters\low\avg_models_low_100.tif') as dem_src:
    raster_file = dem_src.read(1)  

In [79]:
raster_file

array([[-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       ...,
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.]])

In [80]:
for i in range(len(raster_file)):
    for j in range(len(raster_file[1])):
        if raster_file[i][j]>0:
            raster_file[i][j]=1
        else:
            raster_file[i][j]=0

In [81]:
raster_file=raster_file.astype(int)

In [82]:
outputPath=r'zoning_rasters\avg_models_low_100.tif'
cv2.imwrite(outputPath, raster_file)
dataset2 = gdal.Open(outputPath, gdal.GA_Update)
dataset2.SetGeoTransform(geotrans)
dataset2.SetProjection(projection)

0

## High 

In [83]:
df_1_high=func(r'final_rasters\model_1_high')
df_2_high=func(r'final_rasters\model_2_high')
df_3_high=func(r'final_rasters\model_3_high')
df_4_high=func(r'final_rasters\model_4_high')
df_5_high=func(r'final_rasters\model_5_high')

TypeError: func() missing 1 required positional argument: 'jaar'

In [31]:
df_1_high

,fraction inundated,average inundation,name
2,0.087435,0.085682,inunriver_rcp8p5_00000NorESM1-M_2050_rp00002.tif
5,0.098059,0.363708,inunriver_rcp8p5_00000NorESM1-M_2050_rp00005.tif
10,0.106146,0.522222,inunriver_rcp8p5_00000NorESM1-M_2050_rp00010.tif
25,0.1149,0.703681,inunriver_rcp8p5_00000NorESM1-M_2050_rp00025.tif
50,0.118486,0.845044,inunriver_rcp8p5_00000NorESM1-M_2050_rp00050.tif
100,0.121295,0.983044,inunriver_rcp8p5_00000NorESM1-M_2050_rp00100.tif
250,0.124827,1.156996,inunriver_rcp8p5_00000NorESM1-M_2050_rp00250.tif
500,0.127958,1.281797,inunriver_rcp8p5_00000NorESM1-M_2050_rp00500.tif
1000,0.130075,1.415074,inunriver_rcp8p5_00000NorESM1-M_2050_rp01000.tif


In [32]:
df_2_high

,fraction inundated,average inundation,name
2,0.108377,0.538918,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00002.tif
5,0.123056,1.094745,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00005.tif
10,0.130228,1.435217,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00010.tif
25,0.135263,1.87525,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00025.tif
50,0.139536,2.173716,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00050.tif
100,0.144853,2.434164,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00100.tif
250,0.150477,2.781563,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00250.tif
500,0.155255,3.019711,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00500.tif
1000,0.15935,3.256449,inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp01000.tif


In [33]:
df_3_high

,fraction inundated,average inundation,name
2,0.100176,0.400972,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00002.tif
5,0.117438,0.820646,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00005.tif
10,0.123897,1.099268,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00010.tif
25,0.130307,1.464041,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00025.tif
50,0.133878,1.726902,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00050.tif
100,0.136742,1.982965,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00100.tif
250,0.142158,2.278365,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00250.tif
500,0.145699,2.495757,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00500.tif
1000,0.148805,2.710468,inunriver_rcp8p5_0000HadGEM2-ES_2050_rp01000.tif


In [34]:
df_4_high

,fraction inundated,average inundation,name
2,0.100339,0.410236,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00002.tif
5,0.120425,0.943777,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00005.tif
10,0.127745,1.282319,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00010.tif
25,0.133799,1.7186,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00025.tif
50,0.137494,2.037708,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00050.tif
100,0.142351,2.318858,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00100.tif
250,0.148617,2.663035,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00250.tif
500,0.152881,2.913172,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00500.tif
1000,0.157154,3.149276,inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp01000.tif


In [35]:
df_5_high

,fraction inundated,average inundation,name
2,0.08703,0.061994,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00002.tif
5,0.09612,0.350413,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00005.tif
10,0.105824,0.504699,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00010.tif
25,0.114406,0.693042,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00025.tif
50,0.118239,0.839806,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00050.tif
100,0.121122,0.983689,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00100.tif
250,0.125237,1.160122,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00250.tif
500,0.128081,1.28894,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00500.tif
1000,0.129832,1.424054,inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp01000.tif


In [84]:
def func_high(z,jaar):
    all_models=[]
    for y in range(506):
        b= []
        for x in range(398):
            b.append([0,0,0,0,0])
        all_models.append(b)
        
    avg_models_high=[]
    for y in range(506):
        b= []
        for x in range(398):
            b.append(0)
        avg_models_high.append(b)
    avg_models_high=np.array(avg_models_high, dtype = float)

    for count, folder in enumerate(os.listdir(r'final_rasters\high')):
        folder2=os.path.join(r'final_rasters\high',folder)
        file=os.path.join(folder2,os.listdir(folder2)[z])
        print(file)
        if file.endswith('tif'):
            with rio.open(file) as dem_src:
                raster_file = dem_src.read(1)  
                for i in range(506):
                    for j in range(398):
                        all_models[i][j][count]=raster_file[i][j]
                        
    for i in range(506):
        for j in range(398):
            avg_models_high[i][j]=np.mean(all_models[i][j])

    with rio.open(r'mmr_landuse_2016_final.tif') as dem_src:
        raster = gdal.Open(file)
        projection = raster.GetProjection()
        geotrans = raster.GetGeoTransform()
    outputPath = os.path.join(r'combined_rasters','avg_models_high_'+ str(jaar) +'.tif')
    print(outputPath)
    cv2.imwrite(outputPath, avg_models_high)
    dataset2 = gdal.Open(outputPath, gdal.GA_Update)
    dataset2.SetGeoTransform(geotrans)
    dataset2.SetProjection(projection)
    return avg_models_high  

In [85]:
avg_models_high_2=func_high(1,2)
avg_models_high_5=func_high(6,5)
avg_models_high_10=func_high(11,10)
avg_models_high_25=func_high(16,25)
avg_models_high_50=func_high(21,50)
avg_models_high_100=func_high(26,100)
avg_models_high_250=func_high(31,250)
avg_models_high_500=func_high(36,500)
avg_models_high_1000=func_high(41,1000)

final_rasters\high\model_5_high\inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00002.tif
final_rasters\high\model_1_high\inunriver_rcp8p5_00000NorESM1-M_2050_rp00002.tif
final_rasters\high\model_2_high\inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00002.tif
final_rasters\high\model_3_high\inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00002.tif
final_rasters\high\model_4_high\inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00002.tif
combined_rasters\avg_models_high_2.tif
final_rasters\high\model_5_high\inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00005.tif
final_rasters\high\model_1_high\inunriver_rcp8p5_00000NorESM1-M_2050_rp00005.tif
final_rasters\high\model_2_high\inunriver_rcp8p5_0000GFDL-ESM2M_2050_rp00005.tif
final_rasters\high\model_3_high\inunriver_rcp8p5_0000HadGEM2-ES_2050_rp00005.tif
final_rasters\high\model_4_high\inunriver_rcp8p5_00IPSL-CM5A-LR_2050_rp00005.tif
combined_rasters\avg_models_high_5.tif
final_rasters\high\model_5_high\inunriver_rcp8p5_MIROC-ESM-CHEM_2050_rp00010.tif
final_rasters\high\model_1_high

### Export 100-year return period high

In [86]:
file_path=r'mmr_landuse_2016_final.tif'
with rio.open(file_path) as dem_src:
    luc_mmr = dem_src.read(1)
    luc=gdal.Open(file_path)
    projection = luc.GetProjection()
    geotrans = luc.GetGeoTransform()

In [88]:
with rio.open(r'combined_rasters\high\avg_models_high_100.tif') as dem_src:
    raster_file = dem_src.read(1)  

In [89]:
raster_file

array([[-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       ...,
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0.,  0.,  0.]])

In [90]:
for i in range(len(raster_file)):
    for j in range(len(raster_file[1])):
        if raster_file[i][j]>0:
            raster_file[i][j]=1
        else:
            raster_file[i][j]=0

In [91]:
raster_file=raster_file.astype(int)

In [93]:
outputPath=r'zoning_rasters\avg_models_high_100.tif'
cv2.imwrite(outputPath, raster_file)
dataset2 = gdal.Open(outputPath, gdal.GA_Update)
dataset2.SetGeoTransform(geotrans)
dataset2.SetProjection(projection)

0